# Python deltalake 0.9.0: Using Optimize and Vacuum in append-only Delta Lake workloads

Delta Lake tables are an excellent way to store large datasets. With the Python package, you can write ETL jobs that write small amounts of data at a time, when a full distributed cluster might be overkill. This can be useful for workloads that require periodically pulling in data to add to the data lake. However, writing small amounts of data at a time can cause there to be a huge number of files in the table, which can slow down queries.

In deltalake 0.9.0, we added the `optimize` method to the `DeltaTable` class, which performs file compaction. Running this periodically on a table will reduce the number of files in the table, which will speed up queries.

This is very helpful for workloads that append frequently. For example, if you have a table that is appended to every 10 minutes, after a year you will have 52,560 files in the table. If the table is partitioned by another dimension, you will have 52,560 files per partition; with just 100 unique values that's millions of files. By running `optimize` periodically, you can reduce the number of files in the table to a more manageable number.

Typically, you will run optimize less frequently than you append data. If possible, you might run optimize once you know you have finished writing to a particular partition. For example, on a table partitioned by date, you might append data every 10 minutes, but only run optimize once a day at the end of the day. This will ensure you don't need to compact the same data twice.

We'll look at a simple example of this type of workload, and how `optimize` and `vacuum` can be used to improve performance.

In [ ]:
import pyarrow as pa
import pyarrow.compute as pc
from deltalake import DeltaTable, write_deltalake
from datetime import datetime, timedelta
import itertools


: 

To simulate a workload that pulls in new data periodically, we wrote a function that generates a new set of random data given a timestamp. We'll pass a sequence of hours to this, but the frequency could be anything.

In [3]:
def record_observations(date: datetime) -> pa.Table:
    """Pulls data for a certain datetime"""
    nrows = 1000
    return pa.table({
        "date": pa.array([date.date()] * nrows),
        "timestamp": pa.array([date] * nrows),
        "value": pc.random(nrows),
    })

# Example of output
record_observations(datetime(2021, 1, 1, 12)).to_pandas()

,date,timestamp,value
0,2021-01-01,2021-01-01 12:00:00,0.207328
1,2021-01-01,2021-01-01 12:00:00,0.707571
2,2021-01-01,2021-01-01 12:00:00,0.541993
3,2021-01-01,2021-01-01 12:00:00,0.950974
4,2021-01-01,2021-01-01 12:00:00,0.442661
...,...,...,...
995,2021-01-01,2021-01-01 12:00:00,0.248151
996,2021-01-01,2021-01-01 12:00:00,0.256765
997,2021-01-01,2021-01-01 12:00:00,0.282376
998,2021-01-01,2021-01-01 12:00:00,0.643089


First, we'll write 100 hours worth of data to the table.

In [5]:
# Every hour starting at midnight on 2021-01-01
hours_iter = (datetime(2021, 1, 1) + timedelta(hours=i) for i in itertools.count())

# Write 100 hours worth of data
for timestamp in itertools.islice(hours_iter, 100):
    write_deltalake(
        "observation_data",
        record_observations(timestamp),
        partition_by=["date"], #partition by 5 different datetime
        mode="append",
    )
    

We can now load out table's state with `DeltaTable("path/to/table")`. How do we tell how many files there are? We can use the `.files()` method to get the list of files in the current version of the table.

In [6]:
dt = DeltaTable("observation_data")
# We now have 100 files in our table
len(dt.files())

100

We have 100 files, but how many partitions do we have? The `get_add_actions()` method gives us statistics for every file. With a little data wrangling, we can get the unique values for the `date` partition column.

In [8]:
# But there are only 5 unique partitions
dt.get_add_actions(flatten=True).column("partition.date").unique().sort()

[
  2021-01-01,
  2021-01-02,
  2021-01-03,
  2021-01-04,
  2021-01-05
]

Now we can run `optimize()` on our table. This compacts the 100 files into a single file per partition. Since we have 5 partitions, it adds 5 files to the table. The previous 100 files are removed from the table. All this is show in the metrics output by `optimize()`.

In [9]:
dt.optimize()

{'numFilesAdded': 5,
 'numFilesRemoved': 100,
 'filesAdded': {'min': 39000,
  'max': 238282,
  'avg': 198425.6,
  'totalFiles': 5,
  'totalSize': 992128},
 'filesRemoved': {'min': 10244,
  'max': 10244,
  'avg': 10244.0,
  'totalFiles': 100,
  'totalSize': 1024400},
 'partitionsOptimized': 5,
 'numBatches': 1,
 'totalConsideredFiles': 100,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

Now when we check the number of files, we see that we have 5 files, one per partition.

In [10]:
# After running optimize, we have an equal number of files as partitions
len(dt.files())

5

In [16]:
dt.schema()

Schema([Field(date, PrimitiveType("date"), nullable=True), Field(timestamp, PrimitiveType("timestamp"), nullable=True), Field(value, PrimitiveType("double"), nullable=True)])

## Handling incremental updates

Above, we optimized a table when the entire table had too many files. But when we incrementally update the table, we'll only have extra files in the new partitions. Let's take a look at how we handle incremental updates.

We'll add another 24 hours worth of data.

In [11]:
# Add another 24 hours of data
for timestamp in itertools.islice(hours_iter, 24):
    write_deltalake(
        dt,
        record_observations(timestamp),
        partition_by=["date"],
        mode="append",
    )

Now we can use `get_add_actions()` again to introspect the table state. We can see that `2021-01-06` has only a few hours of data so far, so we don't want to optimize that yet. But `2021-01-05` has all 24 hours of data, so it's ready to be optimized.

In [14]:
dt.get_add_actions(flatten=True).to_pandas()["partition.date"].value_counts().sort_index()

partition.date
2021-01-01    1
2021-01-02    1
2021-01-03    1
2021-01-04    1
2021-01-05    1
2021-01-06    4
Name: count, dtype: int64

To optimize a single partition, you can pass in a `partition_filters` argument speficying which partitions to optimize.

In [13]:
dt.optimize(partition_filters=[("date", "=", "2021-01-05")])

{'numFilesAdded': 1,
 'numFilesRemoved': 21,
 'filesAdded': {'min': 238282,
  'max': 238282,
  'avg': 238282.0,
  'totalFiles': 1,
  'totalSize': 238282},
 'filesRemoved': {'min': 10244,
  'max': 39000,
  'avg': 11613.333333333334,
  'totalFiles': 21,
  'totalSize': 243880},
 'partitionsOptimized': 1,
 'numBatches': 1,
 'totalConsideredFiles': 21,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

## Vacuuming after optimizing

When we optimize a table, we remove the old files from the table. However, these files are still in the table's transaction log. This is useful for tables where we might delete data, since we can look at old versions of the table. But for tables where we only append data and optimize partitions, the old files are mostly redundant.

To remove them, we can use the `vacuum()` method. By default, this will remove all files that are older than 7 days. You can pass in a `retention_hours` argument to change this. However, for safety this argument won't allow windows that are too recent, unless you also pass the `enforce_retention_period=False` argument. Since for our workload the old files are redundant, we are okay with `rentention_hours=0`.

In [15]:
dt.vacuum(retention_hours=0, enforce_retention_duration=False, dry_run=False)

['date=2021-01-02/27-45e63e8d-da4a-41e9-a88d-03dbd2d8ace3-0.parquet',
 'date=2021-01-02/39-a2fb8b25-91bb-4733-b10d-98e85cf01ae0-0.parquet',
 'date=2021-01-02/33-02703111-7ee4-441e-85d2-1d115f59c6d1-0.parquet',
 'date=2021-01-02/31-486adea9-f328-4e6a-8db9-4c1a4e417173-0.parquet',
 'date=2021-01-02/38-d6813f30-21c2-4624-a984-ec1ca28c1d28-0.parquet',
 'date=2021-01-02/35-108c3b1c-9617-4a44-9391-1b363f087f74-0.parquet',
 'date=2021-01-02/32-28fae56b-1d07-4bb9-9bc7-d104d3568b72-0.parquet',
 'date=2021-01-02/25-884ec8db-e7da-4715-abd6-b9fc648a90d0-0.parquet',
 'date=2021-01-02/44-61209c2d-ced5-4dd3-bfae-f5fd67356673-0.parquet',
 'date=2021-01-02/36-69da0185-cf54-4f26-a3fe-e56687cf2495-0.parquet',
 'date=2021-01-02/43-9277be65-ef51-46e2-ae96-894c1537623a-0.parquet',
 'date=2021-01-02/24-3ce81b63-a0d9-4a53-80dd-efd5085a03e1-0.parquet',
 'date=2021-01-02/45-f44bbb5c-f9f3-4328-bb21-1b719c6a6036-0.parquet',
 'date=2021-01-02/29-df3fa7d8-7d30-49a5-a2d1-e4cf2fda6151-0.parquet',
 'date=2021-01-02/46